# Dies ist der Klausurtest
Aufgabe: erstelle ein Programm, das die bereitgestellten CSV-Datei einliest
und lowercase am Bildschirm ausgibt, und Zeile f. Zeile mit der jeweiligen Anzahl von Worten
in der jeweiligen Zeile am Anfang.
Bsp: Die Zeile "hEllo World" soll folgendermassen am Bildschirm ausgegeben werden:

2 hello world

In [7]:
from pyspark.sql import SparkSession

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("Klausurtest") \
      .getOrCreate() 

# Einlesen von Dateien in Spark, Datei Typ einfach Anpassen
test=spark.read.csv("../Downloads/Klausurtest.csv")
test.show(truncate=False)

+------------------------------+----+
|_c0                           |_c1 |
+------------------------------+----+
|5 mal werden Wir noch wach    |    |
|Heißa dann ist Weihnachtsnacht|null|
+------------------------------+----+



In [12]:
# Spalten umbenennen für bessere Lesbarkeit
import pyspark.sql.functions as F

spalten = test.select(F.col("_c0").alias("Spalte1"),F.col("_c1").alias("Spalte2"))
spalten.show(truncate=False)

+------------------------------+-------+
|Spalte1                       |Spalte2|
+------------------------------+-------+
|5 mal werden Wir noch wach    |       |
|Heißa dann ist Weihnachtsnacht|null   |
+------------------------------+-------+



In [14]:
# Alles lower machen wie in der Anforderung gewünscht

words_lower=spalten.select(F.lower(F.col("Spalte1")).alias("lower_words"))
words_lower.show(truncate=False)

+------------------------------+
|lower_words                   |
+------------------------------+
|5 mal werden wir noch wach    |
|heißa dann ist weihnachtsnacht|
+------------------------------+



In [23]:
# Wir ersetzen das Sonderzeichen "ß" durch ein einfaches s, da es sonst zu problemen beim weiterverarbeiten kommen könnte
scharfss = words_lower.withColumn('lower_words', F.translate('lower_words', 'ß', 'ss'))
scharfss.show(truncate=False)

+------------------------------+
|lower_words                   |
+------------------------------+
|5 mal werden wir noch wach    |
|heisa dann ist weihnachtsnacht|
+------------------------------+



In [45]:
# Spilten nach Leerzeichen
lines = scharfss.select(F.split(F.col("lower_words"), " ").alias("Zeile"))
lines.show(10,truncate=False)

+-----------------------------------+
|Zeile                              |
+-----------------------------------+
|[5, mal, werden, wir, noch, wach]  |
|[heisa, dann, ist, weihnachtsnacht]|
+-----------------------------------+



In [46]:
# Erstelle eine neue Spalte mit der Anzahl der Worten in der Zeile

anzahl = lines.withColumn("AnzahlWorte",F.size(F.col("Zeile")))
anzahl.show(truncate=False)
anzahl.printSchema()

+-----------------------------------+-----------+
|Zeile                              |AnzahlWorte|
+-----------------------------------+-----------+
|[5, mal, werden, wir, noch, wach]  |6          |
|[heisa, dann, ist, weihnachtsnacht]|4          |
+-----------------------------------+-----------+

root
 |-- Zeile: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- AnzahlWorte: integer (nullable = false)



In [48]:
# Spalten in der gewünschten Reihenfolge anordnen

Dataframe = anzahl.select(F.col("AnzahlWorte"),F.col("Zeile"))
Dataframe.show(truncate=False)

+-----------+-----------------------------------+
|AnzahlWorte|Zeile                              |
+-----------+-----------------------------------+
|6          |[5, mal, werden, wir, noch, wach]  |
|4          |[heisa, dann, ist, weihnachtsnacht]|
+-----------+-----------------------------------+

